In [ ]:
### TESTER DE AUTO-ROCKETPY ###

import auto_rocketpy as sim
import rocket_data as rdata
from datcom import DATCOM
from rocketpy import Environment, SolidMotor, NoseCone, TrapezoidalFins, Rocket
import numpy as np
import datetime

import importlib
importlib.reload(sim)

<module 'simulador' from 'c:\\Users\\kevin\\OneDrive - UPV\\auto-rocketpy\\simulador.py'>

In [2]:
### SE CARGAN LOS DATOS DE LA SIMULACIÓN

data = rdata.load_rocket_data(0)
datcom = DATCOM('D:\\DATCOM')

In [3]:
### DEFINICIÓN DE LA ATMÓSFERA

# Se define la clase Environment
env = Environment(
    latitude = data['latitud_base'],
    longitude = data['longitud_base'],
    elevation = data['elevacion_base']
)

# Perfil lineal del viento
m = (data['viento_mag_10'] - data['viento_mag_0']) / 10000

perfil_viento_u = list()
perfil_viento_v = list()
for h in list(range(0,10000 + 1,30)):
    magnitud_viento = m*h + data['viento_mag_0']  # Ecuación de la recta
    viento_u = magnitud_viento * np.sin( data['viento_heading'] * np.pi/180 ) # Componente dirección Este
    viento_v = magnitud_viento * np.cos( data['viento_heading'] * np.pi/180 ) # COmponente dirección Norte
    perfil_viento_u.append([h,viento_u])
    perfil_viento_v.append([h,viento_v])

# Fecha del lanzamiento
tomorrow = datetime.date.today() + datetime.timedelta(days=1)
env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)

# Se carga el perfil de viento
env.set_atmospheric_model(
    type="custom_atmosphere",
    wind_u = perfil_viento_u,
    wind_v = perfil_viento_v,
)
env.max_expected_height = 4000

In [4]:
### DEFINICIÓN DEL MOTOR

motor = SolidMotor(
    thrust_source = 'validation files/' + data['thrust_source'],
    dry_mass = data['motor_dry_mass'],
    dry_inertia = (
        data['motor_I11'],
        data['motor_I11'],
        data['motor_I33'],
    ),
    nozzle_radius = data['motor_nozzle_radius'],
    grain_number = data['grain_number'],
    grain_density = data['grain_density'],
    grain_outer_radius = data['grain_outer_radius'],
    grain_initial_inner_radius = data['grain_initial_inner_radius'],
    grain_initial_height = data['grain_height'],
    grain_separation = data['grain_separation'],
    grains_center_of_mass_position = data['cdg_grain'],
    center_of_dry_mass_position = data['motor_dry_cdg'],
    nozzle_position = 0,
    throat_radius = data['throat_radius'],
    burn_time = data['burnout_time'],
    coordinate_system_orientation = "nozzle_to_combustion_chamber",
)

In [5]:
### DEFINICIÓN DE LAS SUPERFICIES AERODINÁMICAS

# Nose cone
nose_cone = NoseCone(
    length = data['longitud_nose_cone'],
    kind = data['tipo_nose_cone_rocketpy'],
    base_radius = data['radius'],
    bluffness = None,
    rocket_radius = data['radius'],
    name = "Nose Cone",
)

# Aletas
aletas = TrapezoidalFins(
    n = data['fins_number'],
    root_chord = data['fins_root_chord'],
    tip_chord = data['fins_tip_chord'],
    span = data['fins_span'],
    rocket_radius = data['radius'],
    sweep_length = data['fins_sweep_lenght'],
    name = "Aletas",     
)

In [6]:
# SE DEFINE EL COHETE

rocket = Rocket(
    radius = data['radius'],
    mass = data['no_motor_mass'],
    inertia = (
        data['inercia_I11'],
        data['inercia_I11'],
        data['inercia_I33'],
    ),
    power_off_drag = 0,
    power_on_drag = 0,
    center_of_mass_without_motor = data['cdg_no_motor'],
    coordinate_system_orientation = "nose_to_tail",
)
rocket.add_motor(motor,data['longitud'])
rocket.add_surfaces(nose_cone,0)
rocket.add_surfaces(aletas,data['fins_posicion'])

In [ ]:
## BUCLE DE LA SIMULACIÓN

# Inicialización de la simulación
results,flight = sim.flight_initialization(data,rocket,env)

# Bucle de la simulación
while flight.apogee_time == 0:

    results,flight = sim.flight_simulation(
        data,
        results,
        rocket,
        env,
        flight,
        datcom,
        method='drag',
    )

results.export('auto_drag_test.csv','t','x','y','z','vx','vy','vz','w1','w2','w3','R3')

Archivo 'validation files/auto_drag_test_2.csv' creado con éxito.
